# Import

In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Access API

In [2]:
response = urlopen('https://api.openf1.org/v1/sessions?session_name=Race&year=2023')
data = json.loads(response.read().decode('utf-8'))
df = pd.DataFrame(data)
response = urlopen('https://api.openf1.org/v1/sessions?session_name=Race&year=2024')
data = json.loads(response.read().decode('utf-8'))
df2 = pd.DataFrame(data)
df_sessions = pd.concat([df,df2])

# Create Key List

In [12]:
df_session_keys = df_sessions.drop(columns=['meeting_key', 'location', 'date_start', 'date_end','session_type', 'session_name', 'country_key', 'country_code','country_name', 'circuit_key', 'circuit_short_name', 'gmt_offset',
       'year'])

In [14]:
df_session_keys.to_csv('session_keys.csv', index=False)

# Upload raw_sessions to PSQL

In [4]:
config = dotenv_values()

api_key = config['METEOSTAT_KEY']
pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)